In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from bsread import source
import numpy as np
from scipy.signal import savgol_filter
from scipy.optimize import curve_fit

In [2]:
def pearson_1D_horiz(spectra, I0):
    Pearsons1D_horiz_raw = np.zeros((spectra.shape[1]))
    for i in range(0,spectra.shape[1]):
        coeff = np.corrcoef(spectra[:,i],I0)[0,1]        
        Pearsons1D_horiz_raw[i]= coeff
    return Pearsons1D_horiz_raw

def Spectra_bin_I0(I0, I0_axis, spectra):
    digi = np.digitize(I0, I0_axis)   
    Spec_binned = []
    for i in range(0,len(I0_axis)):
        ind = digi==i+1
        Spec_binned.append(spectra[ind,:].mean(axis=0))
    return np.asarray(Spec_binned)

def prepData(spectra, PBPS053,e_axis,pulseID_shift):
    spectra = spectra[pulseID_shift:]
    e_axis = e_axis[pulseID_shift:]
    PBPS053 = PBPS053[:-pulseID_shift]
    
    allData = np.array([spectra, PBPS053,e_axis])
    indexNone = np.array([np.any(np.hstack(s) == None) for s in allData.T])
    spectra_out = []
    for s in spectra[~indexNone]:
        spectra_out.append(s[:])
    return(np.array(spectra_out), PBPS053[~indexNone],e_axis[~indexNone][0])

## User inputs

In [70]:
channels = ['SARFE10-PSSS059:SPECTRUM_X','SARFE10-PSSS059:SPECTRUM_Y','SARFE10-PBPS053:INTENSITY']
# channels = ['SARFE10-PSSS059:SPECTRUM_X','SARFE10-PSSS059:SPECTRUM_Y','SARFE10-CVME-PHO6212:intensity']

numShots =10000

In [116]:
import data_api as api
import datetime
channels = ['SARFE10-PSSS059:SPECTRUM_X','SARFE10-PSSS059:SPECTRUM_Y','SARFE10-PBPS053:INTENSITY']


In [203]:
now

datetime.datetime(2021, 8, 30, 7, 59, 6, 298838)

In [201]:
now = datetime.datetime.now()
end = now-datetime.timedelta(minutes=1)
start = end-datetime.timedelta(seconds=1)
data = api.get_data(channels=channels, start=start, end=end)

INFO:DataApiClient:Querying channels: ['SARFE10-PSSS059:SPECTRUM_X', 'SARFE10-PSSS059:SPECTRUM_Y', 'SARFE10-PBPS053:INTENSITY']
INFO:DataApiClient:Querying on globalDate between 2021-08-30 07:58:05.298838 and 2021-08-30 07:58:06.298838


In [202]:
# rate of nans
PBPS053_rate = data[channels[2]].isna().sum()/data[channels[2]].shape
PSSS_rate = data[channels[1]].isna().sum()/data[channels[1]].shape
print('PBPS053 rate nans: %.3f'%PBPS053_rate)
print('PSSS rate nans: %.3f'%PSSS_rate)

PBPS053 rate nans: 0.422
PSSS rate nans: 0.578


In [179]:
from bsread import source
channels = ['SARFE10-PSSS059:SPECTRUM_X','SARFE10-PSSS059:SPECTRUM_Y','SARFE10-PBPS053:INTENSITY']
numShots =1000

e_axis = []
spectra = []
PBPS053 = []

with source(channels=channels) as stream:
    for i in range(numShots):
        message = stream.receive()
        e_axis.append(message.data.data[channels[0]].value)
        spectra.append(message.data.data[channels[1]].value)
        PBPS053.append(message.data.data[channels[2]].value)
        
spectra = np.array(spectra)
e_axis = np.array(e_axis)
PBPS053 = np.array(PBPS053)

In [180]:
PBPS053_rate = len(PBPS053[PBPS053 ==None])/len(PBPS053)
PSSS_rate = sum([np.any(s == None) for s in spectra])/len(spectra)
print('PBPS053 rate nans: %.3f'%PBPS053_rate)
print('PSSS rate nans: %.3f'%PSSS_rate)

PBPS053 rate nans: 0.384
PSSS rate nans: 0.478


In [83]:
channels = ['SARFE10-PSSS059:SPECTRUM_X','SARFE10-PSSS059:SPECTRUM_Y','SARFE10-PBPS053:INTENSITY']
numShots =1000

e_axis = []
spectra = []
PBPS053 = []

with source(channels=channels) as stream:
    for i in range(numShots):
        message = stream.receive()
        e_axis.append(message.data.data[channels[0]].value)
        spectra.append(message.data.data[channels[1]].value)
        PBPS053.append(message.data.data[channels[2]].value)
        
spectra_raw = np.array(spectra)
e_axis = np.array(e_axis)
PBPS053_raw = np.array(PBPS053)
# spectra = np.asarray(spectra)
# PBPS053 = np.asarray(PBPS053)

spectra, PBPS053, e_axis = prepData(np.array(spectra), np.array(PBPS053), np.array(e_axis),1)

In [33]:
Pearsons1D_horiz_raw = pearson_1D_horiz(spectra,PBPS053.astype(float))
PBPS053_bin = np.linspace(0.55, 0.95,10)
SpectraBined = Spectra_bin_I0(PBPS053, PBPS053_bin, spectra)

/sf/photo/miniconda/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: Mean of empty slice.
  del sys.path[0]


In [81]:
spectra_raw

array([array([2098, 1687, 2069, ..., 1896, 1848, 1977], dtype=uint32),
       array([1984, 1640, 2021, ..., 1778, 1850, 1861], dtype=uint32),
       array([2157, 1714, 2032, ..., 1846, 1830, 2019], dtype=uint32),
       ..., None, None, None], dtype=object)

In [82]:
for i in spectra_raw:
    if i == None:
        print(i)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [80]:
PBPS053.shape

(172,)

In [ ]:
plt.figure(figsize=[15,10])
plt.suptitle('FEL spectrum and intensity correlation')
plt.subplot(221)
plt.title("Spectrum correlation with PBPS")
plt.plot(e_axis, Pearsons1D_horiz_raw)
plt.xlabel('Photon energy [eV]')

plt.grid(True)
plt.subplot(223)
plt.plot(e_axis, SpectraBined[0,:], label = 'I0 %.2f%%'%PBPS053_bin[0])
plt.plot(e_axis, SpectraBined[-1,:], label = 'I0 %.2f%%'%PBPS053_bin[-1])
plt.plot(e_axis, SpectraBined[int(len(PBPS053_bin)/2),:], label = 'I0 %.2f%%'%PBPS053_bin[int(len(PBPS053_bin)/2)])
plt.xlabel('Photon energy [eV]')

plt.grid(True)
plt.legend()
plt.subplot(122)
plt.pcolormesh(e_axis,PBPS053_bin,SpectraBined, cmap ='CMRmap')
plt.ylabel('I0 (PBPS053)')
plt.xlabel('Photon energy [eV]')
